Creates the database of the toolings by extracting the output file using RyR Generator target files

In [1]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator_personalized, RyR, z_score_filter, reset_df
from utilities_plotting import plot_scatter, plot_capability
import _db_tools as db

Database *input.db* found in: c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_RyR_Analyser\database\input.db
Database *output.db* found in: c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_RyR_Analyser\database\output.db


In [2]:
#File filtering
extension = "xlsx"
file_list = os.listdir("../a1_input/")
filtered_list = [filename for filename in file_list if filename.endswith(extension)]
print("Files ready for storage:")
for file in filtered_list:
    print(f"    {file}")
print("\n")

Files ready for storage:
    TOP_PASSAT_B9_2023y-12m-13d_14h-49m-51s.xlsx




In [3]:
#Database generation
confirmation = input("You are going to modify the database. Are you sure you wrote the correct tooling in the globals file? (y/n): ").strip().lower()
filepaths = []; filepaths_limits = []
if confirmation == 'y':
    for file in filtered_list:
        filepath = os.path.join(os.path.abspath("../a1_input/"), file)
        df = prepare_data(filepath)
        MEAS = prepare_data(filepath, filter="measurements") #Load the output from RyR_Generator into a df
        LIMITS = prepare_data(filepath, filter="limits") #Load the output from RyR_Generator into a df
        file, _ = os.path.splitext(file)
        file = re.sub(r'\W', '_', file)
        prepare_database("input.db", MEAS, file+"_measurements") #Store a df for the measures inside the database of the project
        prepare_database("input.db", LIMITS, file+"_limits") #Store a df for the limits inside the database of the project
        measurements_file = rename_limits_table("input.db", file+"_measurements")
        limits_file = rename_limits_table("input.db", file+"_limits")
        filepaths.append(measurements_file); filepaths_limits.append(limits_file)
    print("Limit files renamed for clarity.")
    consult_database("input.db")
else:
    print("Operation canceled.")

Dataframe stored as *TOP_PASSAT_B9_2023y_12m_13d_14h_49m_51s_measurements*
Dataframe stored as *TOP_PASSAT_B9_2023y_12m_13d_14h_49m_51s_limits*
Limit files renamed for clarity.
*input.db* actual contents:
    TOP_PASSAT_B9_measurements_2023y_12m_13d_14h_49m_51s
    TOP_PASSAT_B9_limits_2023y_12m_13d_14h_49m_51s


In [4]:
#Database management
table_names = [filepaths[0], filepaths_limits[0]]
print("Latest tables: ")
print(table_names)
measurements = retrieve_data("input.db", table_names[0])
limits = retrieve_data("input.db", table_names[1])

Latest tables: 
['TOP_PASSAT_B9_measurements_2023y_12m_13d_14h_49m_51s', 'TOP_PASSAT_B9_limits_2023y_12m_13d_14h_49m_51s']
Table *TOP_PASSAT_B9_measurements_2023y_12m_13d_14h_49m_51s* retrieved succesfully.
Table *TOP_PASSAT_B9_limits_2023y_12m_13d_14h_49m_51s* retrieved succesfully.


In [5]:
###WARNING###
clear_databases(["input.db", "output.db"])